In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Mar 13 03:33:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/SampleDarknet/

/content/drive/MyDrive/SampleDarknet


1. Clone Darknet


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14730, done.
remote: Total 14730 (delta 0), reused 0 (delta 0), pack-reused 14730
Receiving objects: 100% (14730/14730), 13.27 MiB | 8.29 MiB/s, done.
Resolving deltas: 100% (10020/10020), done.
Checking out files: 100% (2023/2023), done.


2. setup darknet to access GPU

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/drive/MyDrive/SampleDarknet/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~

3. Configure Network for training YOLOv3

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!ls cfg

9k.labels				    resnext152-32x4d.cfg
9k.names				    rnn.cfg
9k.tree					    rnn.train.cfg
alexnet.cfg				    strided.cfg
cd53paspp-gamma.cfg			    t1.test.cfg
cifar.cfg				    tiny.cfg
cifar.test.cfg				    tiny-yolo.cfg
coco9k.map				    tiny-yolo-voc.cfg
coco.data				    tiny-yolo_xnor.cfg
coco.names				    vgg-16.cfg
combine9k.data				    vgg-conv.cfg
crnn.train.cfg				    voc.data
csdarknet53-omega.cfg			    writing.cfg
cspx-p7-mish.cfg			    yolo.2.0.cfg
cspx-p7-mish_hp.cfg			    yolo9000.cfg
cspx-p7-mish-omega.cfg			    yolo.cfg
csresnext50-panet-spp.cfg		    yolov1
csresnext50-panet-spp-original-optimal.cfg  yolov2.cfg
darknet19_448.cfg			    yolov2-tiny.cfg
darknet19.cfg				    yolov2-tiny-voc.cfg
darknet53_448_xnor.cfg			    yolov2-voc.cfg
darknet53.cfg				    yolov3_5l.cfg
darknet.cfg				    yolov3.cfg
densenet201.cfg				    yolov3.coco-giou-12.cfg
efficientnet_b0.cfg			    yolov3-openimages.cfg
efficientnet-lite3.cfg			    yolov3-spp.cfg
enet-coco.cfg				    yolov3-tiny_3l

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

create folder to save weight

In [ ]:
!mkdir yoloweight

In [ ]:
!echo "Houseplant" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = yoloweight/' > data/obj.data
!mkdir data/obj

In [ ]:
!ls data/obj
%cat data/obj.data
%cat data/obj.names

classes= 1
train  = data/train.txt
valid  = data/test.txt
names = data/obj.names
backup = yoloweight/
Houseplant


Download weights darknet model 53

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-03-13 03:48:24--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  42.0MB/s    in 4.1s    

2021-03-13 03:48:28 (38.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



Crawling Open Image data set by OID-Toolkits and place image and label in yoloweight folder

In [ ]:
!git clone https://github.com/pythonlessons/OIDv4_ToolKit.git

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 439, done.
remote: Total 439 (delta 0), reused 0 (delta 0), pack-reused 439
Receiving objects: 100% (439/439), 34.09 MiB | 22.96 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [ ]:
%cd OIDv4_ToolKit/

/content/drive/My Drive/SampleDarknet/darknet/OIDv4_ToolKit


In [ ]:
!pip install -r requirements.txt

In [ ]:
!python main.py downloader --classes Houseplant --type_csv train --limit 2500


		   ___   _____  ______            _    _    
		 .'   `.|_   _||_   _ `.         | |  | |   
		/  .-.  \ | |    | | `. \ _   __ | |__| |_  
		| |   | | | |    | |  | |[ \ [  ]|____   _| 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_  
		 `.___.'|_____||______.'   \__/     |_____|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Houseplant.
   [ERROR] | Missing the class-descriptions-boxable.csv file.
[DOWNLOAD] | Do you want to download the missing file? [Y/n] Y
...145%, 0 MB, 70786 KB/s, 0 seconds passed
[DOWNLOAD] | Fi

In [ ]:
%cd ../data/obj/
!ls

/content/drive/My Drive/SampleDarknet/darknet/data/obj
HouseplantCopy.rar


In [ ]:
!unrar x HouseplantCopy.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from HouseplantCopy.rar

Extracting  ffbe5ad1437ca374.jpg                                           0%  OK 
Extracting  ffbe5ad1437ca374.txt                                           0%  OK 
Extracting  ffc32e5434e9176c.jpg                                           0%  OK 
Extracting  ffc32e5434e9176c.txt                                           0%  OK 
Extracting  fff07258b9d2b749.jpg                                           0%  OK 
Extracting  fff07258b9d2b749.txt                                           0%  OK 
Extracting  fff06668422781d1.jpg                                           0%  OK 
Extracting  fff06668422781d1.txt                                           0%  OK 
Extracting  000bdbe22699dea9.jpg                                           0%  OK 
Extracting  000bdbe22699dea9.txt                                   

In [ ]:
!pwd

/content/drive/My Drive/SampleDarknet


In [ ]:
!ls darknet/data/obj

00074ae73610a804.jpg  509f3155543a8fcd.jpg  a93300e10d7ee5e3.jpg
00074ae73610a804.txt  509f3155543a8fcd.txt  a93300e10d7ee5e3.txt
000bdbe22699dea9.jpg  50a7cbc85a2701ff.jpg  a952d78ddf093e39.jpg
000bdbe22699dea9.txt  50a7cbc85a2701ff.txt  a952d78ddf093e39.txt
000cb65712f9360f.jpg  513cbc4f4a6e82d6.jpg  a999372e0933bce0.jpg
000cb65712f9360f.txt  513cbc4f4a6e82d6.txt  a999372e0933bce0.txt
0011748211bcdb1d.jpg  51469bec9989f434.jpg  a9a6fa03e5fd03f5.jpg
0011748211bcdb1d.txt  51469bec9989f434.txt  a9a6fa03e5fd03f5.txt
0015af8d7d6d77c6.jpg  5193cacfb236209b.jpg  a9c039818ee29278.jpg
0015af8d7d6d77c6.txt  5193cacfb236209b.txt  a9c039818ee29278.txt
001f1e83bd627431.jpg  519537a9b2a6d222.jpg  a9d7d0b25e4ee2c4.jpg
001f1e83bd627431.txt  519537a9b2a6d222.txt  a9d7d0b25e4ee2c4.txt
0021d65b07272b1f.jpg  51967f86d5f0208c.jpg  a9da7e9ee6531016.jpg
0021d65b07272b1f.txt  51967f86d5f0208c.txt  a9da7e9ee6531016.txt
0027ea2ab5c65444.jpg  51a50cd45f533e30.jpg  a9f860b2bef6c208.jpg
0027ea2ab5c65444.txt  51a

In [ ]:
#  have been repair by convert OID into Yolov3 Darknet format
# import glob
# import os
# import re

# txt_file_paths = glob.glob(r"data/obj/*.txt")
# for i, file_path in enumerate(txt_file_paths):
#     # get image size
#     with open(file_path, "r") as f_o:
#         lines = f_o.readlines()

#         text_converted = []
#         # print(lines)
#         for line in lines:
#             print(line)
#             numbers = re.findall("[0-9.]+", line)
#             print(numbers)
#             if numbers:

#                 # Define coordinates
#                 text = "{} {} {} {} {}".format(0, numbers[0], numbers[1], numbers[2], numbers[3])
#                 # print(text)
#                 text_converted.append(text)
#                 print(i, file_path)
#                 print(text)
#         # Write file
#         with open(file_path, 'w') as fp:
#             for item in text_converted:
#                 fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/a6093385abcbfeec.jpg', 'data/obj/a62cd7b3f3afac35.jpg', 'data/obj/a63edd1958b2045c.jpg', 'data/obj/a63e9d0c6a088e2c.jpg', 'data/obj/a69819dc7f2af0e7.jpg', 'data/obj/a69995bc3fbb0727.jpg', 'data/obj/a6e7f09cc9ed6b4d.jpg', 'data/obj/a723fec162a43478.jpg', 'data/obj/a75241883e0f9f8e.jpg', 'data/obj/a758140ddd77ed02.jpg', 'data/obj/a75ab2a6a6fb521a.jpg', 'data/obj/a79474dbea4f4797.jpg', 'data/obj/a7a6e2b1aad45710.jpg', 'data/obj/a7b2f838774322c7.jpg', 'data/obj/a7e0d21171f18648.jpg', 'data/obj/a7e583f55ad4c20c.jpg', 'data/obj/a7ef14a8c3072d85.jpg', 'data/obj/a8003674a11417d3.jpg', 'data/obj/a87a9b8adda1ab26.jpg', 'data/obj/a890a4fc1ba7d5ec.jpg', 'data/obj/a8b112d912efb119.jpg', 'data/obj/a8bdd8f5fe38709b.jpg', 'data/obj/a8ed23199e2a99c3.jpg', 'data/obj/a90354780d828910.jpg', 'data/obj/a93300e10d7ee5e3.jpg', 'data/obj/a952d78ddf093e39.jpg', 'data/obj/a999372e0933bce0.jpg', 'data/obj/a9a6fa03e5fd03f5.jpg', 'data/obj/a9c039818ee29278.jpg', 'data/obj/a9d7d0b25e4ee2c4.jpg', 'data/obj

Create training.txt fle


In [ ]:
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

Start training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 587621, rewritten_bbox = 0.190429 % 

 3926: 0.871503, 0.647858 avg loss, 0.001000 rate, 9.988975 seconds, 251264 images, 0.288144 hours left
Loaded: 0.000051 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.832560), count: 7, class_loss = 0.500653, iou_loss = 0.345330, total_loss = 0.845983 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.833242), count: 1, class_loss = 0.279248, iou_loss = 0.012419, total_loss = 0.291667 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000062, iou_loss = 0.000000, total_loss = 0.000062 
 total_bbox = 587629, rewritten_bbox = 0.190426 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.868958), count: 4, class_loss = 0.197698, iou_loss = 0.092597, total_loss = 0.290295 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1

Try realtime object detection

In [ ]:
# %cd darknet/
!pwd

/content/drive/My Drive/SampleDarknet/darknet


In [ ]:
import colorsys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import time


import numpy as np
import tensorflow.python.keras.backend as K
from keras.models import load_model
from keras.layers import Input

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import image_preporcess

import csv
import datetime
import subprocess

class YOLO(object):
	
    _defaults = {
        #"model_path": 'logs/trained_weights_final.h5',
        "model_path": 'model_data/final_model.h5',
        "anchors_path": 'model_data/yolo_anchors.txt',
        "classes_path": 'model_data/houseplant_classes.txt',
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "text_size" : 1,
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()
        

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Memuat model, atau construct model dan load weights
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # settingan default
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate warna untuk bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))

        np.random.shuffle(self.colors)  # Mengacak warna untuk menghias kelas yang berdekatan.

        # Membuat target tensor output untuk kotak pembatas yang difilter
        self.input_image_shape = K.placeholder(shape=(2, ))
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image):
        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = image_preporcess(np.copy(image), tuple(reversed(self.model_image_size)))
            image_data = boxed_image

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.shape[0], image.shape[1]],#[image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        # #print('Found {} boxes for {}'.format(len(out_boxes), 'img'))
        # hitung = int(format(len(out_boxes)))
        # if hitung >= 1:
        # 	increment_value = open("operator.txt","r")
        # 	inc2 = int(increment_value.read())
        # 	increment_value.close()
        # 	inc3 = inc2 + 1
        # 	if inc3 == 5:
        # 		waktu = datetime.datetime.now()
        # 		list1 = [waktu]
        # 		with open("waktu.csv", "a+") as fp:
        # 			wr = csv.writer(fp, dialect="excel")
        # 			wr.writerow(list1)
        # 		subprocess.call(["python3", "test_gpio.py"])
        		
        # 	inc4 = str(inc3)
        # 	print(inc3)
        # 	inc1 = open("operator.txt","w")
        # 	inc1.write(inc4)
        # 	inc1.close()
        	
        

        	
		
        thickness = (image.shape[0] + image.shape[1]) // 600
        fontScale=1
        ObjectsList = []
        
        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)
            #label = '{}'.format(predicted_class)
            scores = '{:.2f}'.format(score)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.shape[0], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.shape[1], np.floor(right + 0.5).astype('int32'))

            mid_h = (bottom-top)/2+top
            mid_v = (right-left)/2+left

            # put object rectangle
            cv2.rectangle(image, (left, top), (right, bottom), self.colors[c], thickness)

            # get text size
            (test_width, text_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, thickness/self.text_size, 1)

            # put text rectangle
            cv2.rectangle(image, (left, top), (left + test_width, top - text_height - baseline), self.colors[c], thickness=cv2.FILLED)

            # put text above rectangle
            cv2.putText(image, label, (left, top-2), cv2.FONT_HERSHEY_SIMPLEX, thickness/self.text_size, (0, 0, 0), 1)

            # add everything to list
            ObjectsList.append([top, left, bottom, right, mid_v, mid_h, label, scores])

        return image, ObjectsList

    def close_session(self):
        self.sess.close()

    def detect_img(self, image):
        #image = cv2.imread(image, cv2.IMREAD_COLOR)
        original_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        original_image_color = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        
        r_image, ObjectsList = self.detect_image(original_image_color)
        return r_image, ObjectsList

    
if __name__=="__main__":
	
    
    initial_value = open("operator.txt", "w")
    initial_value.write("0")
    initial_value.close()
    
    yolo = YOLO()
	
    # set start time to current time
    start_time = time.time()
    # displays the frame rate every 2 second
    display_time = 2
    # Set primarry FPS to 0
    fps = 0

    # we create the video capture object cap
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("We cannot open webcam")

    while True:
        ret, frame = cap.read()
        # resize our captured frame if we need
        frame = cv2.resize(frame, None, fx=1.0, fy=1.0, interpolation=cv2.INTER_AREA)

        # detect object on our frame
        
        r_image, ObjectsList = yolo.detect_img(frame)

        # show us frame with detection
        cv2.imshow("Web cam input", r_image)
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            break

        # calculate FPS
        fps += 1
        TIME = time.time() - start_time
        if TIME > display_time:
            print("FPS:", fps / TIME)
            fps = 0 
            start_time = time.time()


    cap.release()
    cv2.destroyAllWindows()
    yolo.close_session()

model_data/final_model.h5 model, anchors, and classes loaded.


ValueError: ignored